In [ ]:
# !pip install ratsnlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 640.0 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 582.5/582.5 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 65.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 729.2/729.2 kB 48.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 34.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 116.4 MB/s eta 0:00:00


In [ ]:
from google.colab import drive

drive.mount('/gdrive', force_remount = True)

Mounted at /gdrive


# 각종 설정
모델 하이퍼파라메터(hyperparameter)와 저장 위치 등 설정 정보를 선언

인자(argument)의 역할과 내용
- pretrained_model_name : 이전 장에서 파인튜닝한 모델이 사용한 프리트레인 마친 언어모델 이름(단 해당 모델은 허깅페이스 라이브러리에 등록되어 있어야 합니다)
- downstream_model_dir : 이전 장에서 파인튜닝한 모델의 체크포인트 저장 위치.
- max_seq_length : 토큰 기준 입력 문장 최대 길이. 아무 것도 입력하지 않으면 128입니다.

In [ ]:
from ratsnlp.nlpbook.classification import ClassificationDeployArguments

args = ClassificationDeployArguments(
    pretrained_model_name = 'beomi/kcbert-base',
    downstream_model_dir = '/gdrive/MyDrive/nlpbook/checkpoint-doccls',
    max_seq_length = 128,
)

downstream_model_checkpoint_fpath: /gdrive/MyDrive/nlpbook/checkpoint-doccls/epoch=0-val_loss=0.27-v1.ckpt


# 모델 로딩
파인튜닝을 마친 모델과 토크나이저를 읽어 들입니다.

- fine_tuned_model_ckpt['state_dict']: 파인튜닝된 모델의 상태를 가지고 있는 딕셔너리. 모델의 각 레이어에 대한 가중치와 bias 값들 저장
- 'model.classifier.bias': 분류기의 bias 텐서를 가져온다.이 분류기는 각 클래스에 대한 확률을 출력하며, bias 텐서 크기는 분류 클래스 수와 동일
- .shape.numel(): 텐서의 모든 원소의 개수를 반환. 이 경우에는 bias 텐서의 원소의 개수, 즉 클래스의 수를 반환

In [ ]:
import torch
from transformers import BertConfig, BertForSequenceClassification

fine_tuned_model_ckpt = torch.load(
    args.downstream_model_checkpoint_fpath,
    map_location = torch.device('cpu')
)

pretrained_model_config = BertConfig.from_pretrained(
    args.pretrained_model_name,
    num_labels = fine_tuned_model_ckpt['state_dict']['model.classifier.bias'].shape.numel(),
)

model = BertForSequenceClassification(pretrained_model_config)

model.load_state_dict({k.replace('model.', '') : v for k , v in fine_tuned_model_ckpt['state_dict'].items()})

<All keys matched successfully>

In [ ]:
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=0)
      (position_embeddings): Embedding(300, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case = False
)

# 인퍼런스 함수 선언
- 문장(sentence)에 토큰화를 수행한 뒤 input_ids, attention_mask, token_type_ids를 만든다.
- 이들 입력값을 파이토치 텐서(tensor) 자료형으로 변환한 뒤 모델에 입력합니다. - 모델 출력 값(outputs.logits)은 소프트맥스 함수 적용 이전의 로짓(logit) 형태인데요. 여기에 소프트맥스 함수를 써서 모델 출력을 [부정일 확률, 긍정일 확률] 형태의 확률 형태로 바꾼다.
- 마지막으로 모델 출력을 약간 후처리하여 예측 확률의 최댓값이 부정 위치일 경우 해당 문장이 부정(positive), 반대의 경우 긍정(positive)이 되도록 pred 값을 만든다.

In [ ]:
def inference_fn(sentence) :
  inputs = tokenizer(
      [sentence],
      max_length = args.max_seq_length,
      padding = 'max_length',
      truncation = True,
  )

  with torch.no_grad() :
    outputs = model(**{k: torch.tensor(v) for k, v in inputs.items()})
    prob = outputs.logits.softmax(dim = 1)
    positive_prob = round(prob[0][1].item(), 4)
    negative_prob = round(prob[0][0].item(), 4)
    pred = '긍정 (positive)' if torch.argmax(prob) == 1 else '부정 (negtive)'

  return {
      'sentence' : sentence,
      'prediction' : pred,
      'positive_data' : f'긍정 {positive_prob}',
      'negative_data' : f'부정 {negative_prob}',
      'positive_width' : f'{positive_prob * 100}%',
      'negative_width' : f'{round(negative_prob * 100.2)}%'
  }

In [ ]:
sentence = '갈까 말까'

inference_fn(sentence)

{'sentence': '갈까 말까',
 'prediction': '부정 (negtive)',
 'positive_data': '긍정 0.3484',
 'negative_data': '부정 0.6516',
 'positive_width': '34.839999999999996%',
 'negative_width': '65%'}

In [ ]:
sentence = '기린이 그린 기린 그림이 맞는 기린 그린 기린인지 아닌 기린 그린 기린인지 알 수가 없는것 같기도 하고 아닌거 같기도한데\
  어떻게 보면 맞는거 같기도 하고 아닌거 같기도 하고 잘 모르겠다.'

inference_fn(sentence)

{'sentence': '기린이 그린 기린 그림이 맞는 기린 그린 기린인지 아닌 기린 그린 기린인지 알 수가 없는것 같기도 하고 아닌거 같기도한데  어떻게 보면 맞는거 같기도 하고 아닌거 같기도 하고 잘 모르겠다.',
 'prediction': '부정 (negtive)',
 'positive_data': '긍정 0.0133',
 'negative_data': '부정 0.9867',
 'positive_width': '1.3299999999999998%',
 'negative_width': '99%'}

In [ ]:
sentence = '기린이 맞다!'

inference_fn(sentence)

{'sentence': '기린이 맞다!',
 'prediction': '긍정 (positive)',
 'positive_data': '긍정 0.6876',
 'negative_data': '부정 0.3124',
 'positive_width': '68.76%',
 'negative_width': '31%'}

In [ ]:
!mkdir /root/.ngrok2 && echo 'authtoken: 2T5dj5MGKLVe9gdeTmRn96rqUGK_6bLTw98KhQnQENd1C9FCt' > /root/.ngrok2/ngrok.yml

In [ ]:
!ls /root -al

total 76
drwx------ 1 root root 4096 Jul 26 02:08 .
drwxr-xr-x 1 root root 4096 Jul 26 00:46 ..
-r-xr-xr-x 1 root root 1169 Jan  1  2000 .bashrc
drwxr-xr-x 1 root root 4096 Jul 26 00:46 .cache
drwx------ 1 root root 4096 Jul 26 00:42 .config
drwxr-xr-x 5 root root 4096 Jul 20 13:44 .ipython
drwx------ 1 root root 4096 Jul 20 13:44 .jupyter
drwxr-xr-x 2 root root 4096 Jul 20 13:42 .keras
drwx------ 3 root root 4096 Jul 20 13:21 .launchpadlib
drwxr-xr-x 1 root root 4096 Jul 20 13:44 .local
drwxr-xr-x 2 root root 4096 Jul 26 02:08 .ngrok2
drwxr-xr-x 4 root root 4096 Jul 20 13:44 .npm
drwx------ 3 root root 4096 Jul 26 01:06 .nv
-rw-r--r-- 1 root root  161 Jul  9  2019 .profile
-r-xr-xr-x 1 root root  254 Jan  1  2000 .tmux.conf
-rw-r--r-- 1 root root  165 Jul 20 13:44 .wget-hsts


In [ ]:
from ratsnlp.nlpbook.classification import get_web_service_app

app = get_web_service_app(inference_fn)

app.run()

 * Serving Flask app 'ratsnlp.nlpbook.classification.deploy'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://40f8-34-143-226-80.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [26/Jul/2023 02:30:08] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/Jul/2023 02:30:09] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [26/Jul/2023 02:30:14] "POST /api HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/Jul/2023 02:30:21] "POST /api HTTP/1.1" 200 -
